<a href="https://colab.research.google.com/github/FernandooMoraes/Article-Ensemble-of-transfer-learning-models/blob/main/Article_Nodataincrease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pacotes utilizados



In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Input, Dense, ZeroPadding2D,Conv2D, MaxPooling2D, BatchNormalization, Activation
import tensorflow as tf
import cv2
import os
import glob
import skimage.io
import torchvision.models as models
import zipfile
import io
from os import listdir
from sklearn.metrics import confusion_matrix
import imutils
import time
from random import shuffle
from keras import losses
from tensorflow.keras import losses
import shutil
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2 as cv
from PIL import Image, ImageDraw, ImageFilter
from skimage.viewer import ImageViewer
from skimage import filters
from skimage.morphology import erosion, dilation, opening, closing, white_tophat,black_tophat, skeletonize, convex_hull_image,disk
from skimage.data import data_dir
from skimage.util import img_as_ubyte
import copy
from statistics import *
from numpy import array
from google.colab import files
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Upload da base de dados para o colab

In [ ]:
uploaded2 = files.upload()

In [ ]:
data = zipfile.ZipFile('/content/fold0.zip', 'r')
data.extractall()

In [ ]:
rotulos = pd.read_csv("/content/rotulos.csv")

In [ ]:
from google.colab import files
uploaded2 = files.upload()

Saving teste.zip to teste.zip


In [ ]:
data = zipfile.ZipFile('/content/teste.zip', 'r')
data.extractall()

#Leitura da base de dados

In [ ]:
dir_list = ['/content/fold0/train/B/','/content/fold0/train/M/']

In [ ]:
def load_data(dir_list):
    """
    Read images, resize and normalize them.
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []

    for directory in dir_list:
        for filename in sorted(listdir(directory)):
            if filename == '.ipynb_checkpoints':
                print("proximo")
            else:
            # load the image
              image = cv2.imread(directory + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            #image = crop_brain_contour(image, plot=False)
            # resize image
            #image = cv2.resize(image, dsize= (256, 256), interpolation=cv2.INTER_CUBIC)
            # normalize values
              image = image / 255.
            # convert image to numpy array and append it to X
              X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
              if directory == '/content/fold0/train/M/':
                 y.append([1])

              else:
                 y.append([0])

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    #X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

In [ ]:
X_train, y_train = load_data(dir_list)

Number of examples is: 1310
X shape is: (1310, 256, 256, 3)
y shape is: (1310, 1)


In [ ]:
dir_list = ['/content/fold0/test/B/','/content/fold0/test/M/']

In [ ]:
def load_data(dir_list):
    """
    Read images, resize and normalize them.
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []
    y = []

    for directory in dir_list:
        for filename in sorted(listdir(directory)):
            if filename == '.ipynb_checkpoints':
                print("proximo")
            else:
            # load the image
              image = cv2.imread(directory + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            #image = crop_brain_contour(image, plot=False)
            # resize image
            #image = cv2.resize(image, dsize= (256, 256), interpolation=cv2.INTER_CUBIC)
            # normalize values
              image = image / 255.
            # convert image to numpy array and append it to X
              X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
              if directory == '/content/fold0/test/M/':
                 y.append([1])

              else:
                 y.append([0])

    X = np.array(X)
    y = np.array(y)

    # Shuffle the data
    #X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')

    return X, y

In [ ]:
X_test, y_test = load_data(dir_list)

Number of examples is: 90
X shape is: (90, 256, 256, 3)
y shape is: (90, 1)


In [ ]:
dir_list = ['/content/teste/']

In [ ]:
def load_data(dir_list):
    """
    Read images, resize and normalize them.
    Arguments:
        dir_list: list of strings representing file directories.
    Returns:
        X: A numpy array with shape = (#_examples, image_width, image_height, #_channels)
        y: A numpy array with shape = (#_examples, 1)
    """

    # load all images in a directory
    X = []

    for directory in dir_list:
        for filename in listdir(directory):
            if filename == '.ipynb_checkpoints':
                print("proximo")
            else:
            # load the image
              image = cv2.imread(directory + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            #image = crop_brain_contour(image, plot=False)
            # resize image
            #image = cv2.resize(image, dsize= (256, 256), interpolation=cv2.INTER_CUBIC)
            # normalize values
              image = image / 255.
            # convert image to numpy array and append it to X
              X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.

    X = np.array(X)

    # Shuffle the data
    #X, y = shuffle(X, y)

    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')

    return X

In [ ]:
X_test2 = load_data(dir_list)

Number of examples is: 379
X shape is: (379, 256, 256, 3)


In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_test2 = X_test2.astype('float32')
X_train /= 255
X_test /= 255
X_test2 /= 255
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

#Modelo 1

In [ ]:
resultados_test2 = np.zeros((379,13), dtype=np.float64)
resultados = np.zeros((90,13), dtype=np.float64)

In [ ]:
# first input model# first input model
vgg16 = tf.keras.applications.VGG16(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = vgg16.output
x = Flatten()(x)

# merge input models
x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)

model = Model(inputs=vgg16.input, outputs=out)
model.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
#tf_model.summary()

In [ ]:
tf.keras.utils.plot_model(model, "modelo1's.png", show_shapes=True)

In [ ]:
history = model.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = model.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)

In [ ]:
resultados[:,0]=y_predt

In [ ]:
y_predt = model.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)

In [ ]:
resultados_test2[:,0]=y_predt

In [ ]:
with open('modelo1.pkl', 'wb') as file:
    pickle.dump(model, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Modelo2

In [ ]:
densenet121 = tf.keras.applications.densenet.DenseNet121(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = densenet121.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model2=Model(densenet121.input,outputs=out)

tf_model2.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])


In [ ]:
history2 = tf_model2.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model2.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,1]=y_predt
y_predt = tf_model2.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,1]=y_predt

In [ ]:
with open('modelo2.pkl', 'wb') as file:
    pickle.dump(tf_model2, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Modelo 3

In [ ]:
densenet169 = tf.keras.applications.densenet.DenseNet169(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = densenet169.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model3=Model(densenet169.input,outputs=out)

tf_model3.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history3 = tf_model3.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model3.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,2]=y_predt
y_predt = tf_model3.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,2]=y_predt

In [ ]:
with open('modelo3.pkl', 'wb') as file:
    pickle.dump(tf_model3, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Modelo 4

In [ ]:
densenet201 = tf.keras.applications.densenet.DenseNet201(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = densenet201.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model4=Model(densenet201.input,outputs=out)

tf_model4.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history4 = tf_model4.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model4.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,3]=y_predt
y_predt = tf_model4.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,3]=y_predt

In [ ]:
with open('modelo4.pkl', 'wb') as file:
    pickle.dump(tf_model4, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Modelo 5

In [ ]:
inception_resnet_v2 = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = inception_resnet_v2.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model5=Model(inception_resnet_v2.input,outputs=out)

tf_model5.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history5 = tf_model5.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model5.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,4]=y_predt
y_predt = tf_model5.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,4]=y_predt

In [ ]:
with open('modelo5.pkl', 'wb') as file:
    pickle.dump(tf_model5, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Modelo 6

In [ ]:
resnet50 = tf.keras.applications.resnet.ResNet50(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet50.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model6=Model(resnet50.input,outputs=out)

tf_model6.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history6 = tf_model6.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model6.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,5]=y_predt
y_predt = tf_model6.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,5]=y_predt

In [ ]:
with open('modelo6.pkl', 'wb') as file:
    pickle.dump(tf_model6, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 7


In [ ]:
resnet101 = tf.keras.applications.resnet.ResNet101(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet101.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model7=Model(resnet101.input,outputs=out)

tf_model7.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history7 = tf_model7.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model7.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,6]=y_predt
y_predt = tf_model7.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,6]=y_predt

In [ ]:
with open('modelo7.pkl', 'wb') as file:
    pickle.dump(tf_model7, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 8


In [ ]:
resnet152 = tf.keras.applications.resnet.ResNet152(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet152.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model8=Model(resnet152.input,outputs=out)

tf_model8.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history8 = tf_model8.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model8.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,7]=y_predt
y_predt = tf_model8.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,7]=y_predt

In [ ]:
with open('modelo8.pkl', 'wb') as file:
    pickle.dump(tf_model8, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 9

In [ ]:
resnet_50v2 = tf.keras.applications.resnet_v2.ResNet50V2(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet_50v2.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model9=Model(resnet_50v2.input,outputs=out)

tf_model9.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history9 = tf_model9.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model9.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,8]=y_predt
y_predt = tf_model9.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,8]=y_predt

In [ ]:
with open('modelo9.pkl', 'wb') as file:
    pickle.dump(tf_model9, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 10

In [ ]:
resnet_101v2 = tf.keras.applications.resnet_v2.ResNet101V2(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet_101v2.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model10=Model(resnet_101v2.input,outputs=out)

tf_model10.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history10 = tf_model10.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model10.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,9]=y_predt
y_predt = tf_model10.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,9]=y_predt

In [ ]:
with open('modelo10.pkl', 'wb') as file:
    pickle.dump(tf_model10, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 11

In [ ]:
resnet_152v2 = tf.keras.applications.resnet_v2.ResNet101V2(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = resnet_152v2.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model11=Model(resnet_152v2.input,outputs=out)

tf_model11.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history11 = tf_model11.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model11.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,10]=y_predt
y_predt = tf_model11.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,10]=y_predt

In [ ]:
with open('modelo11.pkl', 'wb') as file:
    pickle.dump(tf_model11, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 12

In [ ]:
Xception = tf.keras.applications.xception.Xception(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = Xception.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model12=Model(Xception.input,outputs=out)

tf_model12.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history12 = tf_model12.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model12.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,11]=y_predt
y_predt = tf_model12.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,11]=y_predt

In [ ]:
with open('modelo12.pkl', 'wb') as file:
    pickle.dump(tf_model12, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

#Modelo 13

In [ ]:
MobileNet = tf.keras.applications.mobilenet.MobileNet(input_shape=(256,256,3), include_top = False, weights= 'imagenet')
x = MobileNet.output
x = Flatten()(x)

x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(2,activation='sigmoid')(x)
tf_model13=Model(MobileNet.input,outputs=out)
tf_model13.compile(optimizer=tf.keras.optimizers.Adagrad(),loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
history13 = tf_model13.fit(X_train, y_train, batch_size = 16, epochs = 100, initial_epoch = 0, validation_data = (X_test,y_test),callbacks=EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10))

In [ ]:
y_predt = tf_model13.predict(X_test)
y_predt= np.argmax(y_predt,axis=1)
resultados[:,12]=y_predt
y_predt = tf_model13.predict(X_test2)
y_predt= np.argmax(y_predt,axis=1)
resultados_test2[:,12]=y_predt

In [ ]:
with open('modelo13.pkl', 'wb') as file:
    pickle.dump(tf_model13, file)
np.savetxt('resultadosfold0.csv', resultados, delimiter=',')
np.savetxt('resultados_test2fold0.csv', resultados_test2, delimiter=',')

In [ ]:
files.download('resultados_test2fold0.csv')

In [ ]:
files.download('resultadosfold0.csv')